Your task is to train *character-level* language models. 
You will train unigram, bigram, and trigram character-level models on a collection of books from Project Gutenberg. You will then use these trained English language models to distinguish English documents from Brazilian Portuguese documents in the test set.

In [ ]:
import pandas as pd
import httpimport
import numpy as np
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

with httpimport.remote_repo(['lm_helper'], 'https://raw.githubusercontent.com/jasoriya/CS6120-PS2-support/master/utils/'):
  from lm_helper import get_train_data, get_test_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Unzipping corpora/mac_morpho.zip.


This code loads the training and test data. Each dataset is a list of books. Each book contains a list of sentences, and each sentence contains a list of words. For building a character language model, you should join the words of a sentence together with a space character.

In [ ]:
# get the train and test data
train = get_train_data()
test, test_files = get_test_data()

In [ ]:
# join the words of a sentence together with a space character
def combine(book):
  combine_sentence = []
  for sentence in book:
    combine_sentence.append(" ".join(sentence).strip())
  return combine_sentence

train_list = []
for book in train:
  train_list.append(combine(book))

test_list = []
for book in test:
  test_list.append(combine(book))

## 1.1
Collect statistics on the unigram, bigram, and trigram character counts.

If your machine takes a long time to perform this computation, you may save these counts to files in your github repository and load them on request. This is not necessary, however.

In [ ]:
#@title Default title text
# Your code here
train_set = []
for book in train_list:
  train_set.append(" ".join(book))

test_set = []
for book in test_list:
  test_set.append(" ".join(book))

train_total = [["<s>", "<s>"] + list(sent) + ["</s>", "</s>"] for sent in train_set]

unigram_dict = {}
bigram_dict = {}
trigram_dict = {}

for document in train_total:
  for ch in document:
    if ch in unigram_dict:
      unigram_dict[ch] += 1
    else:
      unigram_dict[ch] = 1
  for i in range(1, len(document)):
    if tuple(document[i-1:i+1]) in bigram_dict:
      bigram_dict[tuple(document[i-1:i+1])] += 1
    else:
      bigram_dict[tuple(document[i-1:i+1])] = 1
  for i in range(2, len(document)):
    if tuple(document[i-2:i+1]) in trigram_dict:
      trigram_dict[tuple(document[i-2:i+1])] += 1
    else:
      trigram_dict[tuple(document[i-2:i+1])] = 1

dict = unigram_dict
for k,v in bigram_dict.items():
  dict[k] = v
for k,v in trigram_dict.items():
  dict[k] = v

## 1.2
Calculate the perplexity for each document in the test set using the linear interpolation smoothing method. For determining λs for linear interpolation of the trigram, bigram, and unigram models, you can divide the training data into a new training set (80%) and a held-out set (20%).
Then choose ~10 random pairs of $(\lambda_3, \lambda_2)$ such that $1 > \lambda_3 > \lambda_2 > 0$ and $\sum_{i=1}^3 \lambda_i = 1$ to test on held-out data.

Some documents in the test set are in Brazilian Portuguese. Identify them as follows: 
  - Sort by perplexity and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names (from `test_files`) and perplexities of the documents above the threshold

    ```
        file name, score
        file name, score
        . . .
        file name, score
    ```

  - Copy this list of filenames and manually annotate them as being correctly or incorrectly labeled as Portuguese.




In [20]:
# Your code here
# divide the training data into a new training set (80%) and a held-out set (20%)
new_training_set, held_out_set = train_test_split(train_set, train_size = 0.8)

# choose ~10 random pairs of  (λ3,λ2)  such that  1>λ3>λ2>0  and  ∑λi=1
def choose_random():
  a, b, c = np.random.rand(3)
  lambda3 = max(a,b,c)/(a+b+c)
  lambda2 = min(a,b,c)/(a+b+c)
  return lambda3, lambda2

lambda_list = []
for i in range(10):
  lambda_list.append(choose_random())

# test on held-out data
held_out = [["<s>", "<s>"] + list(sent) + ["</s>", "</s>"] for sent in held_out_set]
held_out_total = []
for document in held_out:
  held_out_total.extend(document)

V = sum(len(train_total[i]) for i in range(len(train_total)))

p_list = []
for lambda3, lambda2 in lambda_list:
  lambda1 = 1 - lambda3 - lambda2
  p_sum = {}
  for k, v in trigram_dict.items():
    wn_2, wn_1, wn = k
    p_tri = v / bigram_dict[(wn_2, wn_1)]
    p_bi = bigram_dict[(wn_1, wn)] / unigram_dict[wn_1]
    p_uni = unigram_dict[wn] / V
    p_sum[k] = lambda3 * p_tri + lambda2 * p_bi + lambda1 * p_uni
  p_list.append(p_sum)

ppl = []
for i in range(len(p_list)):
  d = p_list[i]
  M = len(held_out_total)
  loss = 0
  for j in range(2, M):
    if tuple(held_out_total[j-2:j+1]) in d:
      loss -= math.log(d[tuple(held_out_total[j-2:j+1])], 2) 
  ppl.append(2**(loss/M-2))

# determining λs
num = ppl.index(min(ppl))
best_lambda = lambda_list[ppl.index(min(ppl))]

# Calculate the perplexity for each document in the test set using the linear interpolation smoothing method
lambda3, lambda2 = best_lambda
score = []
test_doc = []
for document in test_set:
  test_doc.append(["<s>", "<s>"] + list(document) + ["</s>", "</s>"])

for doc in test_doc:
  d = p_list[num]
  M = len(doc)
  loss = 0
  for j in range(2, M):
    if tuple(doc[j-2:j+1]) in d:
      loss -= math.log(d[tuple(doc[j-2:j+1])], 2)
  score.append(2**(loss/M-2))

# Sort by perplexity and set a cut-off threshold, print the file names (from test_files) and perplexities of the documents above the threshold
# set cut-off threshold = 2.5, label them as Portuguese
file_name_score = []
for i in range(len(test_files)):
  file_name_score.append([test_files[i], score[i]])

file_score = sorted(file_name_score, key = lambda x: x[1], reverse = True)
for file in file_score:
  if file[1] > 2.5:
    print(file)
for file in file_score:
  if file[1] > 2.5:
    file.append("Portuguese")
    print(file)



['br94ma01.txt', 3.0927982899804625]
['br94ab02.txt', 3.028269620839364]
['ag94ja11.txt', 3.0266510039402093]
['br94de01.txt', 3.026526711051293]
['br94ju01.txt', 3.0033072696886354]
['ag94ou04.txt', 2.9983251181180055]
['br94jl01.txt', 2.996986170351466]
['br94ag01.txt', 2.977873749170973]
['ag94mr1.txt', 2.9670077201424294]
['ag94fe1.txt', 2.9661849890702294]
['ag94de06.txt', 2.949219423618272]
['br94fe1.txt', 2.9391903121718768]
['ag94ju07.txt', 2.9391552319546226]
['ag94jl12.txt', 2.9280456003106523]
['ag94ab12.txt', 2.923502390178443]
['br94ja04.txt', 2.9211794800983166]
['ag94ma03.txt', 2.9202825789731643]
['ag94ag02.txt', 2.9177032466901442]
['ag94se06.txt', 2.916070350270325]
['ag94no01.txt', 2.907362305167655]
['br94ma01.txt', 3.0927982899804625, 'Portuguese']
['br94ab02.txt', 3.028269620839364, 'Portuguese']
['ag94ja11.txt', 3.0266510039402093, 'Portuguese']
['br94de01.txt', 3.026526711051293, 'Portuguese']
['br94ju01.txt', 3.0033072696886354, 'Portuguese']
['ag94ou04.txt', 2

## 1.3
Build a trigram language model with add-λ smoothing (use λ = 0.1).

Sort the test documents by perplexity and perform a check for Brazilian Portuguese documents as above:

  - Observe the perplexity scores and set a cut-off threshold. All the documents above this threshold score should be categorized as Brazilian Portuguese. 
  - Print the file names and perplexities of the documents above the threshold

  ```
      file name, score
      file name, score
      . . .
      file name, score
  ```

  - Copy this list of filenames and manually annotate them for correctness.

In [19]:
# Your code here
# Build a trigram language model with add-λ smoothing (use λ = 0.1)
score = []
for doc in test_doc:
  loss = 0
  for i in range(2, len(doc)):
    if tuple(doc[i-2:i+1]) not in trigram_dict:
      continue
    p = (trigram_dict[tuple(doc[i-2:i+1])] + 0.1) / (bigram_dict[tuple(doc[i-2:i])] + 0.1 * len(unigram_dict))
    loss -= math.log(p, 2)
  score.append(2**(loss/len(doc)-2))

# Sort the test documents by perplexity. Observe the perplexity scores and set a cut-off threshold.
# set cut-off threshold = 4, label them as Portuguese
file_name_score = []
for i in range(len(test_files)):
  file_name_score.append([test_files[i], score[i]])

file_score = sorted(file_name_score, key = lambda x: x[1], reverse = True)
for file in file_score:
  if file[1] > 4:
    print(file)

for file in file_score:
  if file[1] > 4:
    file.append("portuguese")
    print(file)


['br94ma01.txt', 4.708558765400146]
['ag94ja11.txt', 4.535695915049895]
['ag94mr1.txt', 4.522396813150476]
['br94de01.txt', 4.473970834056714]
['ag94ou04.txt', 4.46607394500025]
['br94ab02.txt', 4.457455444806291]
['br94ag01.txt', 4.453764697290207]
['ag94fe1.txt', 4.447371632716189]
['br94ju01.txt', 4.438187067144307]
['br94jl01.txt', 4.430159927623639]
['ag94de06.txt', 4.429197241597126]
['ag94ju07.txt', 4.419240086236285]
['ag94se06.txt', 4.37705679524243]
['ag94jl12.txt', 4.3659380631017815]
['ag94ma03.txt', 4.348022038772641]
['ag94no01.txt', 4.321492194256158]
['ag94ag02.txt', 4.320988220185967]
['ag94ab12.txt', 4.313528118956486]
['br94fe1.txt', 4.297262402987829]
['br94ja04.txt', 4.290974032358516]
['br94ma01.txt', 4.708558765400146, 'portuguese']
['ag94ja11.txt', 4.535695915049895, 'portuguese']
['ag94mr1.txt', 4.522396813150476, 'portuguese']
['br94de01.txt', 4.473970834056714, 'portuguese']
['ag94ou04.txt', 4.46607394500025, 'portuguese']
['br94ab02.txt', 4.457455444806291, 

## 1.4
Based on your observation from above questions, compare linear interpolation and add-λ smoothing by listing out their pros and cons.

[Your text here.]  
Based on my observation from the results of linear interpolation and add-λ smoothing, both the methods can classify the Engilish and Brazilian Portuguese by seting cut-off thresholds. 
The pros of add-λ smoothing: 
1. The model is simpler to build，  
2. The threshold is easier to find.
3. We could avoid conditional probility equals to 0.   

The cons of add-λ smoothing:  
1. The model is not used in N-grams.
2. Only for text classification.
3. The zero counts are not too many.

Then pros of linear interpolation:
1. The model can be used in N-grams.
2. The model can be trained.
3. Most times linear interpolation works better.
4. Easy to use

The cons of linear interpolation: 
1. The model is not very accurate for non-linear.
2. The accuracy is not good as that of add-λ smoothing.
3. The threshold is harder to find.